In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import dask.dataframe as dd

import warnings
warnings.filterwarnings('ignore')

Load the data

In [33]:
#df = pd.read_csv('../csv/credit_card_fraud_2019_q1.csv')
df = pd.read_csv('csv/credit_card_fraud_sample_10K.csv')

In [34]:
# Set Pandas options to display full rows
pd.set_option('display.max_colwidth', None)  # Show full column contents
pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.width', 1000)  # Adjust width for full display
#pd.set_option('display.max_rows', None)  # Show all rows (use with caution for large datasets)

In [37]:
df.head()

,Unnamed: 0,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long
0,8208382,421-07-7177,3534945553430195,Shelia,Fleming,F,153 Dennis Crossing,Weston,WV,26452,39.0411,-80.4737,9676,Human resources officer,1965-12-28,541742634590,adults_50up_female_urban.json,5b159948cb3f4e2b1a82dbe014e2f7db,2020-06-04,19:55:30,1591280730,health_fitness,16.74,0,fraud_Hirthe-Beier,38.224419,-80.100552
1,12606453,061-53-6686,4183314058259431806,Nicole,Todd,F,088 Jeffrey Prairie Apt. 394,Mountain Center,CA,92561,33.6401,-116.5567,1661,"Administrator, charities/voluntary organisations",1967-05-14,554244831767,adults_50up_female_rural.json,93550825285d02993ed8a30d269025c8,2019-11-10,01:16:24,1573328784,shopping_net,745.63,0,fraud_Schmidt and Sons,32.952717,-116.368019
2,28301366,038-31-5036,2261044046656441,Matthew,Mann,M,3315 Holmes Junctions,Greensboro,NC,27410,36.1032,-79.8794,305168,Oncologist,1996-09-06,628245054586,adults_2550_male_urban.json,b6a1fc16622b0ed557f0af093ecc8512,2020-08-12,20:29:53,1597244393,home,94.13,0,"fraud_Bahringer, Bergnaum and Quitzon",35.419988,-80.107734
3,11771739,830-11-6351,374795838353160,Carolyn,Hall,F,242 Carroll Bypass,Spokane,WA,99208,47.7374,-117.4352,342856,Hydrologist,1962-07-31,663905650306,adults_50up_female_urban.json,2e944bdc8faa762005f0b0486010d307,2019-09-21,02:28:11,1569013091,shopping_net,73.43,0,"fraud_Heathcote, Yost and Kertzmann",47.441771,-117.255467
4,12450008,408-99-8998,4122635746026142330,Andre,Rice,M,8085 Burgess Junctions,Myrtle Beach,SC,29588,33.7226,-78.9782,119080,"Therapist, occupational",2003-06-30,115066483855,young_adults_male_urban.json,f4c5d40c54cb268beb8f501df22785d6,2019-04-21,03:00:46,1555795846,gas_transport,35.49,0,"fraud_Raynor, Reinger and Hagenes",33.911297,-78.057010


In [38]:
df.isna().sum()

Unnamed: 0    0
ssn           0
cc_num        0
first         0
last          0
gender        0
street        0
city          0
state         0
zip           0
lat           0
long          0
city_pop      0
job           0
dob           0
acct_num      0
profile       0
trans_num     0
trans_date    0
trans_time    0
unix_time     0
category      0
amt           0
is_fraud      0
merchant      0
merch_lat     0
merch_long    0
dtype: int64

Clean the Data
1. consider drop first/last name, ssn, cc_num, trans_num etc...
2. explore locational data e.g zip lat/long, city etc... TBD replace with regions??
3. reorganize date & time data:
    3.1 work day vs weekend
    3.2 TBD: split hour segment, work/sleep/free hours
4. set 4 age groups youngest to oldest (18-199), teenager (-18) young_adult (19-25), adult (26-65), retired (...)
    4.1 TBD treat different for men & women?? retirement age 66(M), 63(F)

5. categorize: category,          

6. explore: amt, dob? 


In [39]:
df.nunique()


Unnamed: 0    10000
ssn            7155
cc_num         7155
first           635
last            957
gender            2
street         7155
city           3030
state            51
zip            5348
lat            5237
long           5306
city_pop       3535
job             639
dob            6130
acct_num       7155
profile          12
trans_num     10000
trans_date      731
trans_time     9382
unix_time      9999
category         14
amt            6608
is_fraud          2
merchant        693
merch_lat      9996
merch_long    10000
dtype: int64

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10000 non-null  int64  
 1   ssn         10000 non-null  object 
 2   cc_num      10000 non-null  int64  
 3   first       10000 non-null  object 
 4   last        10000 non-null  object 
 5   gender      10000 non-null  object 
 6   street      10000 non-null  object 
 7   city        10000 non-null  object 
 8   state       10000 non-null  object 
 9   zip         10000 non-null  int64  
 10  lat         10000 non-null  float64
 11  long        10000 non-null  float64
 12  city_pop    10000 non-null  int64  
 13  job         10000 non-null  object 
 14  dob         10000 non-null  object 
 15  acct_num    10000 non-null  int64  
 16  profile     10000 non-null  object 
 17  trans_num   10000 non-null  object 
 18  trans_date  10000 non-null  object 
 19  trans_time  10000 non-null

In [41]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [42]:
print(df['gender'].value_counts())
df['gender'] = df['gender'].map({'M': 1, 'F': 0})
print(df['gender'].value_counts())


gender
F    5184
M    4816
Name: count, dtype: int64
gender
0    5184
1    4816
Name: count, dtype: int64


In [45]:

df['trans_date'] = pd.to_datetime(df['trans_date'])
df['trans_time'] = pd.to_datetime(df['trans_time'])

df['trans_year'] = df['trans_date'].dt.year
df['trans_month'] = df['trans_date'].dt.month
df['trans_day'] = df['trans_date'].dt.day

df['trans_hour'] = df['trans_time'].dt.hour
df['trans_minute'] =df['trans_time'].dt.minute
#df['trans_second'] =df['trans_time'].dt.second


df['dob'] = pd.to_datetime(df['dob'])
df['age'] = df['trans_year']  - df['dob'].dt.year


In [46]:
#age groups

def age_group(age):
    if age < 15:
        return 'child'
    elif age < 20:
        return 'teenager'
    elif age < 40:
        return 'young_adult'
    elif age < 60:
        return 'middle_aged'
    elif age < 80:
        return 'senior'
    else:
        return 'elderly'

df['age_group'] = df['age'].apply(age_group)

In [47]:
df['age_group'].value_counts()

age_group
young_adult    4378
middle_aged    3098
senior         1277
teenager        774
elderly         360
child           113
Name: count, dtype: int64

In [48]:
df['ssn'] = df['ssn'].astype('string')
df['street'] = df['street'].astype('string')
df['city'] = df['city'].astype('string')
df['state'] = df['state'].astype('string')
df['profile'] = df['profile'].astype('string')
df['trans_num'] = df['trans_num'].astype('string')
df['category'] = df['category'].astype('string')
df['merchant'] = df['merchant'].astype('string')



In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            10000 non-null  int64         
 1   ssn           10000 non-null  string        
 2   cc_num        10000 non-null  int64         
 3   first         10000 non-null  object        
 4   last          10000 non-null  object        
 5   gender        10000 non-null  int64         
 6   street        10000 non-null  string        
 7   city          10000 non-null  string        
 8   state         10000 non-null  string        
 9   zip           10000 non-null  int64         
 10  lat           10000 non-null  float64       
 11  long          10000 non-null  float64       
 12  city_pop      10000 non-null  int64         
 13  job           10000 non-null  object        
 14  dob           10000 non-null  datetime64[ns]
 15  acct_num      10000 non-null  int64  

In [50]:
# set area
#df['profile'] = df['profile'].str.replace('.json', '') 
# Extract the area (last word before .json)
df['area'] = df['profile'].str.extract(r'_(rural|urban)\.json')

print(df['area'].value_counts())


area
urban    9478
rural     522
Name: count, dtype: Int64


In [ ]:
#potential dropouts
#['ssn','cc_num', 'trans_num', 'first','last','dob']


In [51]:
df.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long,trans_year,trans_month,trans_day,trans_hour,trans_minute,age,age_group,area
0,8208382,421-07-7177,3534945553430195,Shelia,Fleming,0,153 Dennis Crossing,Weston,WV,26452,39.0411,-80.4737,9676,Human resources officer,1965-12-28,541742634590,adults_50up_female_urban.json,5b159948cb3f4e2b1a82dbe014e2f7db,2020-06-04,2025-03-09 19:55:30,1591280730,health_fitness,16.74,0,fraud_Hirthe-Beier,38.224419,-80.100552,2020,6,4,19,55,55,middle_aged,urban
1,12606453,061-53-6686,4183314058259431806,Nicole,Todd,0,088 Jeffrey Prairie Apt. 394,Mountain Center,CA,92561,33.6401,-116.5567,1661,"Administrator, charities/voluntary organisations",1967-05-14,554244831767,adults_50up_female_rural.json,93550825285d02993ed8a30d269025c8,2019-11-10,2025-03-09 01:16:24,1573328784,shopping_net,745.63,0,fraud_Schmidt and Sons,32.952717,-116.368019,2019,11,10,1,16,52,middle_aged,rural
2,28301366,038-31-5036,2261044046656441,Matthew,Mann,1,3315 Holmes Junctions,Greensboro,NC,27410,36.1032,-79.8794,305168,Oncologist,1996-09-06,628245054586,adults_2550_male_urban.json,b6a1fc16622b0ed557f0af093ecc8512,2020-08-12,2025-03-09 20:29:53,1597244393,home,94.13,0,"fraud_Bahringer, Bergnaum and Quitzon",35.419988,-80.107734,2020,8,12,20,29,24,young_adult,urban
3,11771739,830-11-6351,374795838353160,Carolyn,Hall,0,242 Carroll Bypass,Spokane,WA,99208,47.7374,-117.4352,342856,Hydrologist,1962-07-31,663905650306,adults_50up_female_urban.json,2e944bdc8faa762005f0b0486010d307,2019-09-21,2025-03-09 02:28:11,1569013091,shopping_net,73.43,0,"fraud_Heathcote, Yost and Kertzmann",47.441771,-117.255467,2019,9,21,2,28,57,middle_aged,urban
4,12450008,408-99-8998,4122635746026142330,Andre,Rice,1,8085 Burgess Junctions,Myrtle Beach,SC,29588,33.7226,-78.9782,119080,"Therapist, occupational",2003-06-30,115066483855,young_adults_male_urban.json,f4c5d40c54cb268beb8f501df22785d6,2019-04-21,2025-03-09 03:00:46,1555795846,gas_transport,35.49,0,"fraud_Raynor, Reinger and Hagenes",33.911297,-78.057010,2019,4,21,3,0,16,teenager,urban
